In [199]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [224]:
stage_s = list(np.arange(2,32,2))+list(np.arange(32,48,4))+[50]
stage_s_i = list(np.arange(1,21,1))
stage_s_dict = dict(zip( stage_s_i, stage_s[::-1]))

In [203]:
stage_gc = list(np.arange(2,12,2))
stage_gc_i = list(np.arange(1,6,1))
stage_gc_dict = dict(zip( stage_gc_i, stage_gc[::-1]))

{1: 10, 2: 8, 3: 6, 4: 4, 5: 2}

In [204]:
stage_points = [1]+list(np.arange(2,10,2))
stage_points_i = list(np.arange(1,6,1))
points_dict = dict(zip( stage_points_i,  stage_points[::-1]))

{1: 8, 2: 6, 3: 4, 4: 2, 5: 1}

In [205]:
stage_kom = [1]+list(np.arange(2,10,2))
stage_kom_i = list(np.arange(1,6,1))
kom_dict = dict(zip( stage_kom_i,  stage_kom[::-1]))

{1: 8, 2: 6, 3: 4, 4: 2, 5: 1}

In [206]:
stage_youth = list(np.arange(1,5,1))+[6]
stage_youth_i = list(np.arange(1,6,1))
youth_dict = dict(zip( stage_youth_i,  stage_youth[::-1]))
youth_dict

{1: 6, 2: 4, 3: 3, 4: 2, 5: 1}

In [208]:
gc_s = list(np.arange(2,26,2))+list(np.arange(28,44,4))+[50, 60, 80, 100]
gc_s_i = list(np.arange(1,21,1))
gc_s_dict = dict(zip( gc_s_i, gc_s[::-1]))

{1: 100,
 2: 80,
 3: 60,
 4: 50,
 5: 40,
 6: 36,
 7: 32,
 8: 28,
 9: 24,
 10: 22,
 11: 20,
 12: 18,
 13: 16,
 14: 14,
 15: 12,
 16: 10,
 17: 8,
 18: 6,
 19: 4,
 20: 2}

In [209]:
points_s = list(np.arange(2,10,2))+list(np.arange(10,50,10))+[60, 80]
points_s_i = list(np.arange(1,11,1))
points_s_dict = dict(zip( points_s_i, points_s[::-1]))

{1: 80, 2: 60, 3: 40, 4: 30, 5: 20, 6: 10, 7: 8, 8: 6, 9: 4, 10: 2}

In [210]:
kom_s = list(np.arange(2,10,2))+list(np.arange(10,50,10))+[60, 80]
kom_s_i = list(np.arange(1,11,1))
kom_s_dict = dict(zip( kom_s_i, kom_s[::-1]))

{1: 80, 2: 60, 3: 40, 4: 30, 5: 20, 6: 10, 7: 8, 8: 6, 9: 4, 10: 2}

In [211]:
youth_s = [10,20,30,40,60]
youth_s_i = list(np.arange(1,6,1))
youth_s_dict = dict(zip( youth_s_i, youth_s[::-1]))

{1: 60, 2: 40, 3: 30, 4: 20, 5: 10}

In [228]:
def get_points(race):
    for year in [2016, 2017, 2018, 2019, 2020, 2021]:
        zestien = pd.read_csv(f'data_raw_{str(year)}_{race}.csv', index_col=0).rename(columns={'index': 'rider'})
        
        stages_scorito = zestien.copy()
        stages_scorito['sum_stages'] = 0
        for i in stages_scorito.columns:
            if '-stage' in i:
                stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                stages_scorito[i] = stages_scorito[i].map(stage_s_dict).fillna(0)
                stages_scorito['sum_stages'] = stages_scorito[i] + stages_scorito['sum_stages']
            elif '-gc' in i:
                stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                stages_scorito[i] = stages_scorito[i].map(stage_gc_dict).fillna(0)
                stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
            elif '-points' in i:
                stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                stages_scorito[i] = stages_scorito[i].map(points_dict).fillna(0)
                stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
            elif '-youth' in i:
                stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                stages_scorito[i] = stages_scorito[i].map(youth_dict).fillna(0)
                stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
            elif '-kom' in i:
                stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                stages_scorito[i] = stages_scorito[i].map(kom_dict).fillna(0)
                stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
            else:
                pass
        stages_filt = stages_scorito[['rider', 'sum_stages']]
        
        gc_scorito = zestien.copy()
        gc_scorito['sum_gc'] = 0
        for i in gc_scorito.columns:
            if '21-gc' in i:
                gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                gc_scorito[i] = gc_scorito[i].map(gc_s_dict).fillna(0)
                gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
            elif '21-points' in i:
                gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                gc_scorito[i] = gc_scorito[i].map(points_s_dict).fillna(0)
                gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
            elif '21-youth' in i:
                gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                gc_scorito[i] = gc_scorito[i].map(youth_s_dict).fillna(0)
                gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
            elif '21-kom' in i:
                gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                gc_scorito[i] = gc_scorito[i].map(kom_s_dict).fillna(0)
                gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
            else:
                pass
        gc_filt = gc_scorito[['rider', 'sum_gc']]
        
        teams_scorito = zestien.copy()
        teams_scorito['sum_teams'] = 0
        for i in teams_scorito.columns:
            if 'stage' in i:
                teams_scorito[i] = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_scorito[teams_scorito[i]==1]['team'].item()
                team_index = teams_scorito[teams_scorito['team'] == team][i].index
                not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
                teams_scorito[i][team_index] = 10
                teams_scorito[i][not_team_index] = 0
                teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
            elif '-gc' in i:
                teams_scorito[i] = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_scorito[teams_scorito[i]==1]['team'].item()
                team_index = teams_scorito[teams_scorito['team'] == team][i].index
                not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
                teams_scorito[i][team_index] = 8
                teams_scorito[i][not_team_index] = 0
                teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
            elif '-points' in i:
                teams_scorito[i]  = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_scorito[teams_scorito[i]==1]['team'].item()
                team_index = teams_scorito[teams_scorito['team'] == team][i].index
                not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
                teams_scorito[i][team_index] = 6
                teams_scorito[i][not_team_index] = 0
                teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
            elif '-youth' in i:
                teams_scorito[i]  = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_scorito[teams_scorito[i]==1]['team'].item()
                team_index = teams_scorito[teams_scorito['team'] == team][i].index
                not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
                teams_scorito[i][team_index] = 3
                teams_scorito[i][not_team_index] = 0
                teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
            elif '-kom' in i:
                teams_scorito[i]  = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_scorito[teams_scorito[i]==1]['team'].item()
                team_index = teams_scorito[teams_scorito['team'] == team][i].index
                not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
                teams_scorito[i][team_index] = 6
                teams_scorito[i][not_team_index] = 0
                teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
            else:
                pass
        team_filt = teams_scorito[['rider', 'sum_teams']]
        
        teams_gc_scorito = zestien.copy()
        teams_gc_scorito['sum_gc_teams'] = 0
        for i in teams_gc_scorito.columns:
            if '21-gc' in i:
                teams_gc_scorito[i] = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
                team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
                not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
                teams_gc_scorito[i][team_index] = 24
                teams_gc_scorito[i][not_team_index] = 0
                teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
            elif '21-points' in i:
                teams_gc_scorito[i]  = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
                team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
                not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
                teams_gc_scorito[i][team_index] = 18
                teams_gc_scorito[i][not_team_index] = 0
                teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
            elif '21-youth' in i:
                teams_gc_scorito[i]  = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
                team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
                not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
                teams_gc_scorito[i][team_index] = 9
                teams_gc_scorito[i][not_team_index] = 0
                teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
            elif '21-kom' in i:
                teams_gc_scorito[i]  = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).replace('DSQ', 0).replace('\xa0DNS\xa0', 0).fillna(0).astype('int64')
                team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
                team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
                not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
                teams_gc_scorito[i][team_index] = 18
                teams_gc_scorito[i][not_team_index] = 0
                teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
            else:
                pass
        team_gc_filt = teams_gc_scorito[['rider', 'sum_gc_teams']]
        
        points = stages_filt.merge(gc_filt, how='left', on='rider').merge(team_filt, how='left', on='rider').merge(team_gc_filt, how='left', on='rider')
        scorito_df = pd.DataFrame(points.set_index('rider').sum(axis=1)).reset_index().rename(columns={0:'scorito'}).sort_values(by='scorito', ascending=False)
        scorito_df['year'] = year
        
        scorito_df.to_csv(f'data/data_points_{str(year)}.csv')
        

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/2452504912.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_scorito[i][team_index] = 10
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/2452504912.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_scorito[i][not_team_index] = 0
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/2452504912.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [212]:
gc_scorito = zestien.copy()
gc_scorito['sum_gc'] = 0
for i in gc_scorito.columns:
    if '21-gc' in i:
        gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        gc_scorito[i] = gc_scorito[i].map(gc_s_dict).fillna(0)
        gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
    elif '21-points' in i:
        gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        gc_scorito[i] = gc_scorito[i].map(points_s_dict).fillna(0)
        gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
    elif '21-youth' in i:
        gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        gc_scorito[i] = gc_scorito[i].map(youth_s_dict).fillna(0)
        gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
    elif '21-kom' in i:
        gc_scorito[i] = gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        gc_scorito[i] = gc_scorito[i].map(kom_s_dict).fillna(0)
        gc_scorito['sum_gc'] = gc_scorito['sum_gc']+ gc_scorito[i]
    else:
        pass
gc_filt = gc_scorito[['rider', 'sum_gc']]

In [213]:
teams_scorito = zestien.copy()
teams_scorito['sum_teams'] = 0
for i in teams_scorito.columns:
    if 'stage' in i:
        teams_scorito[i] = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_scorito[teams_scorito[i]==1]['team'].item()
        team_index = teams_scorito[teams_scorito['team'] == team][i].index
        not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
        teams_scorito[i][team_index] = 10
        teams_scorito[i][not_team_index] = 0
        teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
    elif '-gc' in i:
        teams_scorito[i] = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_scorito[teams_scorito[i]==1]['team'].item()
        team_index = teams_scorito[teams_scorito['team'] == team][i].index
        not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
        teams_scorito[i][team_index] = 8
        teams_scorito[i][not_team_index] = 0
        teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
    elif '-points' in i:
        teams_scorito[i]  = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_scorito[teams_scorito[i]==1]['team'].item()
        team_index = teams_scorito[teams_scorito['team'] == team][i].index
        not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
        teams_scorito[i][team_index] = 6
        teams_scorito[i][not_team_index] = 0
        teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
    elif '-youth' in i:
        teams_scorito[i]  = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_scorito[teams_scorito[i]==1]['team'].item()
        team_index = teams_scorito[teams_scorito['team'] == team][i].index
        not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
        teams_scorito[i][team_index] = 3
        teams_scorito[i][not_team_index] = 0
        teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
    elif '-kom' in i:
        teams_scorito[i]  = teams_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_scorito[teams_scorito[i]==1]['team'].item()
        team_index = teams_scorito[teams_scorito['team'] == team][i].index
        not_team_index = teams_scorito[teams_scorito['team'] != team][i].index
        teams_scorito[i][team_index] = 6
        teams_scorito[i][not_team_index] = 0
        teams_scorito['sum_teams'] = teams_scorito['sum_teams']+ teams_scorito[i]
    else:
        pass
team_filt = teams_scorito[['rider', 'sum_teams']]

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/360589024.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_scorito[i][team_index] = 10
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/360589024.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_scorito[i][not_team_index] = 0
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/360589024.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [214]:
teams_gc_scorito = zestien.copy()
teams_gc_scorito['sum_gc_teams'] = 0
for i in teams_gc_scorito.columns:
    if '21-gc' in i:
        teams_gc_scorito[i] = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
        team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
        not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
        teams_gc_scorito[i][team_index] = 24
        teams_gc_scorito[i][not_team_index] = 0
        teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
    elif '21-points' in i:
        teams_gc_scorito[i]  = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
        team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
        not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
        teams_gc_scorito[i][team_index] = 18
        teams_gc_scorito[i][not_team_index] = 0
        teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
    elif '21-youth' in i:
        teams_gc_scorito[i]  = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
        team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
        not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
        teams_gc_scorito[i][team_index] = 9
        teams_gc_scorito[i][not_team_index] = 0
        teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
    elif '21-kom' in i:
        teams_gc_scorito[i]  = teams_gc_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        team = teams_gc_scorito[teams_gc_scorito[i]==1]['team'].item()
        team_index = teams_gc_scorito[teams_gc_scorito['team'] == team][i].index
        not_team_index = teams_gc_scorito[teams_gc_scorito['team'] != team][i].index
        teams_gc_scorito[i][team_index] = 18
        teams_gc_scorito[i][not_team_index] = 0
        teams_gc_scorito['sum_gc_teams'] = teams_gc_scorito['sum_gc_teams']+ teams_gc_scorito[i]
    else:
        pass
team_gc_filt = teams_gc_scorito[['rider', 'sum_gc_teams']]

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/2570851797.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_gc_scorito[i][team_index] = 24
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/2570851797.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_gc_scorito[i][not_team_index] = 0
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_63733/2570851797.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [ ]:
stages_scorito = zestien.copy()
stages_scorito['sum_stages'] = 0
for i in stages_scorito.columns:
    if '-stage' in i:
        stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        stages_scorito[i] = stages_scorito[i].map(stage_s_dict).fillna(0)
        stages_scorito['sum_stages'] = stages_scorito[i] + stages_scorito['sum_stages']
    elif '-gc' in i:
        stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        stages_scorito[i] = stages_scorito[i].map(stage_gc_dict).fillna(0)
        stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
    elif '-points' in i:
        stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        stages_scorito[i] = stages_scorito[i].map(points_dict).fillna(0)
        stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
    elif '-youth' in i:
        stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        stages_scorito[i] = stages_scorito[i].map(youth_dict).fillna(0)
        stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
    elif '-kom' in i:
        stages_scorito[i] = stages_scorito[i].replace('DNS', 0).replace('DNF', 0).replace('OTL',0).fillna(0).astype('int64')
        stages_scorito[i] = stages_scorito[i].map(kom_dict).fillna(0)
        stages_scorito['sum_stages'] = stages_scorito['sum_stages']+ stages_scorito[i]
    else:
        pass
stages_filt = stages_scorito[['rider', 'sum_stages']]